In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import radvel

import rvsearch

%matplotlib inline

/Users/lee/anaconda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [9]:
metadata = pd.read_csv('../legacy_starlist.csv')

In [11]:
filename = 'legacy_search.sh'
legacy_search = open(filename, 'w+')
for name in metadata['name']:
    legacy_search.write('python search_away.py \'{}\' \n'.format(name))

/Users/lee/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: ResourceWarning: unclosed file <_io.TextIOWrapper name='legacy_search.sh' mode='w+' encoding='UTF-8'>
  


In [12]:
names = metadata['name'].tolist()
len(names)

775

In [11]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
file_len('legacy_search.sh')

775

In [11]:
test = scrape(done)

In [12]:
test

[{'star': '1326',
  'num_planets': 1,
  'per1': 11.44249922805173,
  'tc1': -5.747149562318226,
  'secosw1': -0.13470370626650957,
  'sesinw1': 0.1355450732264826,
  'k1': 2.931246731066709,
  'gamma_j': 0.6626024831790128,
  'jit_k': 2.6698605079031945,
  'jit_apf': 1.1043236277294453,
  'gamma_apf': 1.932825696942148,
  'gamma_k': 1.1932088279252633,
  'jit_j': 1.9649490467490995,
  'dvdt': -0.0008883548323190264,
  'curv': 4.38622433361013e-08},
 {'star': '42618',
  'num_planets': 1,
  'per1': 2813.154238349782,
  'tc1': 1515.419164524498,
  'secosw1': 0.03821660864298257,
  'sesinw1': -0.7245105060616197,
  'k1': 2.8959163750662373,
  "gamma_b'k'": 3.1216949473726547,
  "gamma_b'j'": -2.6576441530426798,
  "jit_b'j'": 2.7177022900911187,
  "jit_b'k'": 3.8365278225272474,
  'dvdt': 0.001405874764913878,
  'curv': -6.831252527689229e-08},
 {'star': '95128',
  'num_planets': 2,
  'per1': 1087.1728762539049,
  'tc1': 655.1160865195303,
  'secosw1': -0.009662999780191243,
  'sesinw1': -

# Compute planet masses and semi-major axes.

In [2]:
massdata = pd.read_csv('../legacy_with_masses_mod.csv')

In [7]:
starlist = ['1326','42618','95128','114783','128311','143761','168009','190360']

In [25]:
def scrape(starlist, mass_db_name=None, save=True):
    """This takes data from completed searches and compiles it into one database.
    """
    all_params = []
    nplanets = []

    for star in starlist:
        params = {}
        params['name'] = star
        try:
            post = radvel.posterior.load(star+'/post_final.pkl')
        except RuntimeError:
            print('I am not done looking for planets around {} yet, \
                                try again later.'.format(star))
            continue
            
        if post.params.num_planets == 1:
            if post.params['k1'].value == 0.:
                num_planets = 0
            else:
                num_planets = 1
            nplanets.append(num_planets)
        else:
            num_planets = post.params.num_planets
            nplanets.append(num_planets)
        params['num_planets'] = num_planets

        for k in post.params.keys():
            params[k] = post.params[k].value
        all_params.append(params)

    # Save radvel parameters as a pandas dataframe.
    props = pd.DataFrame(all_params)

    if mass_db_name is not None:
        try:
            mass_db = pd.read_csv(mass_db_name)
        except RuntimeError:
            print('That is not a pandas dataframe. Try again.')
        
        # Add enough columns to compute masses & semi-major axes for system with most planets.
        max_num_planets = np.amax(nplanets)
        for n in np.arange(1, max_num_planets+1):
            props['Mstar'] = np.nan
            props['uMstar1'] = np.nan
            props['uMstar1'] = np.nan
            props['M{}'.format(n)] = np.nan
            props['a{}'.format(n)] = np.nan

        # Save median star mass, uncertainties
        for star in starlist:
            star_index = props.index[props['name'] == str(star)][0]
            mass_index = mass_db.index[mass_db['name'] == str(star)][0]
            # Save star mass, to be used in planet mass & semi-major axis calculations.
            Mtot = mass_db.loc[mass_index, 'mstar']
            props.loc[star_index, 'Mstar'] = Mtot
            props.loc[star_index, 'uMstar1'] = mass_db.loc[mass_index, 'umass1']
            props.loc[star_index, 'uMstar1'] = mass_db.loc[mass_index, 'umass2']
            
            # For each found planet, compute mass and semi-major axis
            if props.loc[star_index, 'num_planets'] != 0:
                for n in np.arange(1, props.loc[star_index, 'num_planets']+1):
                    K = props.loc[star_index, 'k{}'.format(n)]
                    P = props.loc[star_index, 'per{}'.format(n)]
                    e = props.loc[star_index, 'secosw{}'.format(n)]**2 + props.loc[star_index, 'sesinw{}'.format(n)]**2
                    props.loc[star_index, 'M{}'.format(n)] = radvel.utils.Msini(K, P, Mtot, e, Msini_units='jupiter')
                    props.loc[star_index, 'a{}'.format(n)] = radvel.utils.semi_major_axis(P, Mtot)

    if save:
        props.to_csv('system_props.csv')
    return props

In [24]:
system_props = scrape(starlist, 'legacy_with_masses.csv')